# Take off
First `objdump` the "bomb"

```
objdump -d bomb > bomb.S
```

then search for `main` section, here we are

```
0000000000400da0 <main>:
  400da0:	53                   	push   %rbx
  400da1:	83 ff 01             	cmp    $0x1,%edi
  400da4:	75 10                	jne    400db6 <main+0x16>
  400da6:	48 8b 05 9b 29 20 00 	mov    0x20299b(%rip),%rax        # 603748 <stdin@GLIBC_2.2.5>
  ...
```

there are several key word here: read_line, phase_x, phase_defused. Ok, now half way to success.




# Landing

## Context 
The program use `puts` to show the text, whose def is `int puts(const char *s);`, so `mov    $0x??????,%edi` below actually means the pointer arg of the `str` text.
```
  400e19:	e8 84 05 00 00       	call   4013a2 <initialize_bomb> # init bomb
  400e1e:	bf 38 23 40 00       	mov    $0x402338,%edi
  400e23:	e8 e8 fc ff ff       	call   400b10 <puts@plt>        # show text
  400e28:	bf 78 23 40 00       	mov    $0x402378,%edi
  400e2d:	e8 de fc ff ff       	call   400b10 <puts@plt>        # show text
  400e32:	e8 67 06 00 00       	call   40149e <read_line>
  400e37:	48 89 c7             	mov    %rax,%rdi
  400e3a:	e8 a1 00 00 00       	call   400ee0 <phase_1>
  400e3f:	e8 80 07 00 00       	call   4015c4 <phase_defused>
```

## phase 1
before look into `phase_1`, wtf are its args?
```
0000000000400ee0 <phase_1>:                                                       # %rdi = return addr of read_line
  400ee0:	48 83 ec 08          	sub    $0x8,%rsp
  400ee4:	be 00 24 40 00       	mov    $0x402400,%esi                     # %rsi = 0x402400
  400ee9:	e8 4a 04 00 00       	call   401338 <strings_not_equal>
  400eee:	85 c0                	test   %eax,%eax
  400ef0:	74 05                	je     400ef7 <phase_1+0x17>
  400ef2:	e8 43 05 00 00       	call   40143a <explode_bomb>
  400ef7:	48 83 c4 08          	add    $0x8,%rsp
  400efb:	c3                   	ret    
```

let's step into `strings_not_equal`

```
0000000000401338 <strings_not_equal>:
  401338:	41 54                	push   %r12
  40133a:	55                   	push   %rbp
  40133b:	53                   	push   %rbx
  40133c:	48 89 fb             	mov    %rdi,%rbx                          # ret of read_line in %rdi=%rbx
  40133f:	48 89 f5             	mov    %rsi,%rbp                          # 0x402400 in %rsi=%rbp
  401342:	e8 d4 ff ff ff       	call   40131b <string_length>             # now %rax = len(ret of read_line)
  401347:	41 89 c4             	mov    %eax,%r12d                         # %r12d = %rax = len(ret of read_line)
  40134a:	48 89 ef             	mov    %rbp,%rdi                          # %rdi = 0x402400
  40134d:	e8 c9 ff ff ff       	call   40131b <string_length>             # %rax = len(whatever in 0x402400)
  401352:	ba 01 00 00 00       	mov    $0x1,%edx                          # %rdx = 0x1
  401357:	41 39 c4             	cmp    %eax,%r12d                         
  40135a:	75 3f                	jne    40139b <strings_not_equal+0x63>    # jne -> return 1 -> indicating not equal
  ...
  40139b:	89 d0                	mov    %edx,%eax
  40139d:	5b                   	pop    %rbx
  40139e:	5d                   	pop    %rbp
  40139f:	41 5c                	pop    %r12
  4013a1:	c3                   	ret   
```

Alright, now let's look into that miserable cunt behind $0x402400$, use gdb `x/s 0x402400`, we got "Border relations with Canada have never been better.", phase_1 solved.

## phase 2

the addr if %rsp is a mass below, so we simpily assume it as $x$.

```
0000000000400efc <phase_2>:                                                       # %rdi = ret of read_line    
  400efc:	55                   	push   %rbp                                
  400efd:	53                   	push   %rbx
  400efe:	48 83 ec 28          	sub    $0x28,%rsp                         # %rsp = x - 0x28 = x - 40
  400f02:	48 89 e6             	mov    %rsp,%rsi                          # %rsi = x - 40
  400f05:	e8 52 05 00 00       	call   40145c <read_six_numbers>          # %rdi = ret of read_line, %rsi = x-40
  400f0a:	83 3c 24 01          	cmpl   $0x1,(%rsp)                        # start number is 1
  400f0e:	74 20                	je     400f30 <phase_2+0x34>
  400f10:	e8 25 05 00 00       	call   40143a <explode_bomb>
  400f15:	eb 19                	jmp    400f30 <phase_2+0x34>
  400f17:	8b 43 fc             	mov    -0x4(%rbx),%eax
  400f1a:	01 c0                	add    %eax,%eax                          # double every time move to next number.
  400f1c:	39 03                	cmp    %eax,(%rbx)
  400f1e:	74 05                	je     400f25 <phase_2+0x29>
  400f20:	e8 15 05 00 00       	call   40143a <explode_bomb>
  400f25:	48 83 c3 04          	add    $0x4,%rbx
  400f29:	48 39 eb             	cmp    %rbp,%rbx
  400f2c:	75 e9                	jne    400f17 <phase_2+0x1b>
  400f2e:	eb 0c                	jmp    400f3c <phase_2+0x40>
  400f30:	48 8d 5c 24 04       	lea    0x4(%rsp),%rbx
  400f35:	48 8d 6c 24 18       	lea    0x18(%rsp),%rbp
  400f3a:	eb db                	jmp    400f17 <phase_2+0x1b>
  400f3c:	48 83 c4 28          	add    $0x28,%rsp
  400f40:	5b                   	pop    %rbx
  400f41:	5d                   	pop    %rbp
  400f42:	c3                   	ret    
```

```
000000000040145c <read_six_numbers>:                                              # numbers are in %rsp->%rsp+24, 6 number in total, each hold 4 byte  
  40145c:	48 83 ec 18          	sub    $0x18,%rsp                         # %rsp = x - 40 - 0x18 = x - 64
  401460:	48 89 f2             	mov    %rsi,%rdx                          # %rdx = %rsi = %rsp + 24
  401463:	48 8d 4e 04          	lea    0x4(%rsi),%rcx                     # %rcx = x - 40 + 0x4 = x - 36
  401467:	48 8d 46 14          	lea    0x14(%rsi),%rax                    # %rax = x - 40 + 0x14 = x - 20
  40146b:	48 89 44 24 08       	mov    %rax,0x8(%rsp)                     # |%rsp+16 <-> %rsp+8| = %rax = x - 20
  401470:	48 8d 46 10          	lea    0x10(%rsi),%rax                    # %rax = x - 40 + 0x10 = x - 24
  401474:	48 89 04 24          	mov    %rax,(%rsp)                        # |%rsp+8 <-> %rsp| = %rax = x - 24
  401478:	4c 8d 4e 0c          	lea    0xc(%rsi),%r9                      # %r9 = x - 40 + 0xc = x - 28
  40147c:	4c 8d 46 08          	lea    0x8(%rsi),%r8                      # %r9 = x - 40 + 0x8 = x - 32
  401480:	be c3 25 40 00       	mov    $0x4025c3,%esi                     # %rsi = 0x4025c3, should be format str point
  401485:	b8 00 00 00 00       	mov    $0x0,%eax                          # %rax = 0
  40148a:	e8 61 f7 ff ff       	call   400bf0 <__isoc99_sscanf@plt>
  40148f:	83 f8 05             	cmp    $0x5,%eax
  401492:	7f 05                	jg     401499 <read_six_numbers+0x3d>
  401494:	e8 a1 ff ff ff       	call   40143a <explode_bomb>
  401499:	48 83 c4 18          	add    $0x18,%rsp
  40149d:	c3                   	ret   
```

Ok, it is not too hard, input "1 2 4 8 16 32" to pass.

## phase 3

```
0000000000400f43 <phase_3>:
  400f43:	48 83 ec 18          	sub    $0x18,%rsp
  400f47:	48 8d 4c 24 0c       	lea    0xc(%rsp),%rcx
  400f4c:	48 8d 54 24 08       	lea    0x8(%rsp),%rdx
  400f51:	be cf 25 40 00       	mov    $0x4025cf,%esi
  400f56:	b8 00 00 00 00       	mov    $0x0,%eax
  400f5b:	e8 90 fc ff ff       	call   400bf0 <__isoc99_sscanf@plt>
  400f60:	83 f8 01             	cmp    $0x1,%eax                          # ok, this time we have two number. stored in %rsp+8, %rsp+12 respectively.
  400f63:	7f 05                	jg     400f6a <phase_3+0x27>
  400f65:	e8 d0 04 00 00       	call   40143a <explode_bomb>
  400f6a:	83 7c 24 08 07       	cmpl   $0x7,0x8(%rsp)                     
  400f6f:	77 3c                	ja     400fad <phase_3+0x6a>              # num1 in %rsp+8 should be less or equal to 7
  400f71:	8b 44 24 08          	mov    0x8(%rsp),%eax     
  400f75:	ff 24 c5 70 24 40 00 	jmp    *0x402470(,%rax,8)                 # jump to deref addr (0x402470+8*num1) and aligned to byte 8
  # -------------------- condition branch ----------------------
  400f7c:	b8 cf 00 00 00       	mov    $0xcf,%eax
  400f81:	eb 3b                	jmp    400fbe <phase_3+0x7b>
  400f83:	b8 c3 02 00 00       	mov    $0x2c3,%eax
  400f88:	eb 34                	jmp    400fbe <phase_3+0x7b>
  400f8a:	b8 00 01 00 00       	mov    $0x100,%eax
  400f8f:	eb 2d                	jmp    400fbe <phase_3+0x7b>
  400f91:	b8 85 01 00 00       	mov    $0x185,%eax
  400f96:	eb 26                	jmp    400fbe <phase_3+0x7b>
  400f98:	b8 ce 00 00 00       	mov    $0xce,%eax
  400f9d:	eb 1f                	jmp    400fbe <phase_3+0x7b>
  400f9f:	b8 aa 02 00 00       	mov    $0x2aa,%eax
  400fa4:	eb 18                	jmp    400fbe <phase_3+0x7b>
  400fa6:	b8 47 01 00 00       	mov    $0x147,%eax
  400fab:	eb 11                	jmp    400fbe <phase_3+0x7b>
  # ------------------------------------------------------------
  400fad:	e8 88 04 00 00       	call   40143a <explode_bomb>
  400fb2:	b8 00 00 00 00       	mov    $0x0,%eax
  400fb7:	eb 05                	jmp    400fbe <phase_3+0x7b>
  400fb9:	b8 37 01 00 00       	mov    $0x137,%eax
  400fbe:	3b 44 24 0c          	cmp    0xc(%rsp),%eax
  400fc2:	74 05                	je     400fc9 <phase_3+0x86>
  400fc4:	e8 71 04 00 00       	call   40143a <explode_bomb>
  400fc9:	48 83 c4 18          	add    $0x18,%rsp
  400fcd:	c3                   	ret    
```
Let's display the byte code arround $0x402470$. Bytes addrs are wrapped with |< ADDR >|. Wow, it coressponds to the addr above, bravo!
Seems we have more than one correct answer.
```
  402470:	|< 7c 0f                	jl     402481 <_IO_stdin_used+0x1d1>
  402472:	40 00 00             	rex add %al,(%rax)
  402475:	00 00                	add    %al,(%rax)
  402477:	00 >|< b9 0f 40 00 00    	add    %bh,0x400f(%rcx)
  40247d:	00 00                	add    %al,(%rax)
  40247f:	00 >|< 83 0f 40 00 00    	add    %al,0x400f(%rbx)
  402485:	00 00                	add    %al,(%rax)
  402487:	00 >|< 8a 0f 40 00 00    	add    %cl,0x400f(%rdx)
  40248d:	00 00                	add    %al,(%rax)
  40248f:	00 >|< 91 0f 40 00 00    	add    %dl,0x400f(%rcx)
  402495:	00 00                	add    %al,(%rax)
  402497:	00 >|< 98 0f 40 00 00    	add    %bl,0x400f(%rax)
  40249d:	00 00                	add    %al,(%rax)
  40249f:	00 >|< 9f 0f 40 00 00    	add    %bl,0x400f(%rdi)
  4024a5:	00 00                	add    %al,(%rax)
  4024a7:	00 >|< a6 0f 40 00 00    	add    %ah,0x400f(%rsi)
  4024ad:	00 00 >|               	add    %al,(%rax)
```
I pick when $(\%rsp+8) = 0$, $(\%rsp+12) = 0xcf$, which is "0 207". Halfway, baby!

## phase 4

```
000000000040100c <phase_4>:
  40100c:	48 83 ec 18          	sub    $0x18,%rsp
  401010:	48 8d 4c 24 0c       	lea    0xc(%rsp),%rcx
  401015:	48 8d 54 24 08       	lea    0x8(%rsp),%rdx
  40101a:	be cf 25 40 00       	mov    $0x4025cf,%esi
  40101f:	b8 00 00 00 00       	mov    $0x0,%eax
  401024:	e8 c7 fb ff ff       	call   400bf0 <__isoc99_sscanf@plt>
  401029:	83 f8 02             	cmp    $0x2,%eax
  40102c:	75 07                	jne    401035 <phase_4+0x29>              # input has two num, asssume them as num1, num2
  40102e:	83 7c 24 08 0e       	cmpl   $0xe,0x8(%rsp)
  401033:	76 05                	jbe    40103a <phase_4+0x2e>              # num1 <= 0xe
  401035:	e8 00 04 00 00       	call   40143a <explode_bomb>
  40103a:	ba 0e 00 00 00       	mov    $0xe,%edx
  40103f:	be 00 00 00 00       	mov    $0x0,%esi
  401044:	8b 7c 24 08          	mov    0x8(%rsp),%edi
  401048:	e8 81 ff ff ff       	call   400fce <func4>                     # func4(num1, 0, 0xe) 
  40104d:	85 c0                	test   %eax,%eax             
  40104f:	75 07                	jne    401058 <phase_4+0x4c>              # the ret of func4 must be 0
  401051:	83 7c 24 0c 00       	cmpl   $0x0,0xc(%rsp)
  401056:	74 05                	je     40105d <phase_4+0x51>              # num2 should be 0
  401058:	e8 dd 03 00 00       	call   40143a <explode_bomb>
  40105d:	48 83 c4 18          	add    $0x18,%rsp
  401061:	c3 
```
look into func4:

```
0000000000400fce <func4>:
  400fce:	48 83 ec 08          	sub    $0x8,%rsp
  400fd2:	89 d0                	mov    %edx,%eax                          # %rax = 0xe
  400fd4:	29 f0                	sub    %esi,%eax                          # %rax = 0xe
  400fd6:	89 c1                	mov    %eax,%ecx                          # %rcx = 0xe
  400fd8:	c1 e9 1f             	shr    $0x1f,%ecx                         # %rcx = 0
  400fdb:	01 c8                	add    %ecx,%eax                          # %rax = 0xe
  400fdd:	d1 f8                	sar    %eax                               # %rax = 0x7
  400fdf:	8d 0c 30             	lea    (%rax,%rsi,1),%ecx                 # %rcx = 0x7
  400fe2:	39 f9                	cmp    %edi,%ecx                          
  400fe4:	7e 0c                	jle    400ff2 <func4+0x24>                # [jump if num1 >= %rcx]
  400fe6:	8d 51 ff             	lea    -0x1(%rcx),%edx                    # [num1 < %rcx] %rdx -= 1 
  400fe9:	e8 e0 ff ff ff       	call   400fce <func4>                     # recursive
  400fee:	01 c0                	add    %eax,%eax    
  400ff0:	eb 15                	jmp    401007 <func4+0x39>
  400ff2:	b8 00 00 00 00       	mov    $0x0,%eax                          # [num1 >= %rcx] %rax = 0  <= should be this where ret 0
  400ff7:	39 f9                	cmp    %edi,%ecx
  400ff9:	7d 0c                	jge    401007 <func4+0x39>                # [jump if num1 <= %rcx], ret
  400ffb:	8d 71 01             	lea    0x1(%rcx),%esi                     # *num1 > 0x6*
  400ffe:	e8 cb ff ff ff       	call   400fce <func4>
  401003:	8d 44 00 01          	lea    0x1(%rax,%rax,1),%eax
  401007:	48 83 c4 08          	add    $0x8,%rsp
  40100b:	c3                   	ret    
```

So only when $\%rcx<=num1<=\%rcx$, return is valid. The ans is "7 0".

## phase 5
```
0000000000401062 <phase_5>:
  401062:	53                   	push   %rbx
  401063:	48 83 ec 20          	sub    $0x20,%rsp
  401067:	48 89 fb             	mov    %rdi,%rbx
  40106a:	64 48 8b 04 25 28 00 	mov    %fs:0x28,%rax
  401071:	00 00 
  401073:	48 89 44 24 18       	mov    %rax,0x18(%rsp)
  401078:	31 c0                	xor    %eax,%eax
  40107a:	e8 9c 02 00 00       	call   40131b <string_length>             # get length 
  40107f:	83 f8 06             	cmp    $0x6,%eax                          
  401082:	74 4e                	je     4010d2 <phase_5+0x70>              # must be length of 6
  401084:	e8 b1 03 00 00       	call   40143a <explode_bomb>
  401089:	eb 47                	jmp    4010d2 <phase_5+0x70>
  40108b:	0f b6 0c 03          	movzbl (%rbx,%rax,1),%ecx                 # [second jump] %rcx = *(%rbx+%rax)
  40108f:	88 0c 24             	mov    %cl,(%rsp)
  401092:	48 8b 14 24          	mov    (%rsp),%rdx                        # char in str in %rdx
  401096:	83 e2 0f             	and    $0xf,%edx                          # need least significant 4 bits
  401099:	0f b6 92 b0 24 40 00 	movzbl 0x4024b0(%rdx),%edx                
  4010a0:	88 54 04 10          	mov    %dl,0x10(%rsp,%rax,1)    
  4010a4:	48 83 c0 01          	add    $0x1,%rax
  4010a8:	48 83 f8 06          	cmp    $0x6,%rax
  4010ac:	75 dd                	jne    40108b <phase_5+0x29>
  4010ae:	c6 44 24 16 00       	movb   $0x0,0x16(%rsp)
  4010b3:	be 5e 24 40 00       	mov    $0x40245e,%esi                     # "flyers" in 0x40245e
  4010b8:	48 8d 7c 24 10       	lea    0x10(%rsp),%rdi                    # "flyers" => 0x10(%rsp) = 66 6c 79 65 72 73 
  4010bd:	e8 76 02 00 00       	call   401338 <strings_not_equal>
  4010c2:	85 c0                	test   %eax,%eax
  4010c4:	74 13                	je     4010d9 <phase_5+0x77>
  4010c6:	e8 6f 03 00 00       	call   40143a <explode_bomb>
  4010cb:	0f 1f 44 00 00       	nopl   0x0(%rax,%rax,1)
  4010d0:	eb 07                	jmp    4010d9 <phase_5+0x77>
  4010d2:	b8 00 00 00 00       	mov    $0x0,%eax                          # [first jump] %rax = 0
  4010d7:	eb b2                	jmp    40108b <phase_5+0x29>
  4010d9:	48 8b 44 24 18       	mov    0x18(%rsp),%rax
  4010de:	64 48 33 04 25 28 00 	xor    %fs:0x28,%rax
  4010e5:	00 00 
  4010e7:	74 05                	je     4010ee <phase_5+0x8c>
  4010e9:	e8 42 fa ff ff       	call   400b30 <__stack_chk_fail@plt>
  4010ee:	48 83 c4 20          	add    $0x20,%rsp
  4010f2:	5b                   	pop    %rbx
  4010f3:	c3                   	ret    
```
flyers => 66 6c 79 65 72 73 => 9 f e 5 6 7
```
00000000004024b0 <array.3449>:
  4024b0:	6d                   	insl   (%dx),%es:(%rdi)
  4024b1:	61                   	(bad)  
  4024b2:	64 75 69             	fs jne 40251e <array.3449+0x6e>
  4024b5:	65 72 73             	gs jb  40252b <array.3449+0x7b>
  4024b8:	6e                   	outsb  %ds:(%rsi),(%dx)
  4024b9:	66 6f                	outsw  %ds:(%rsi),(%dx)
  4024bb:	74 76                	je     402533 <array.3449+0x83>
  4024bd:	62                   	(bad)  
  4024be:	79 6c                	jns    40252c <array.3449+0x7c>
```
So answer is "\[i/y\] \[o\] \[n\] \[e/u\] \[f/v\] \[g/w\]", any capital is allowed, non-alphabet is also allowed as long as the least significant 4 bits follow "9 f e 5 6 7". I use "Ionuvw".

## phase 6
```
00000000004010f4 <phase_6>:
  4010f4:	41 56                	push   %r14
  4010f6:	41 55                	push   %r13
  4010f8:	41 54                	push   %r12
  4010fa:	55                   	push   %rbp
  4010fb:	53                   	push   %rbx
  4010fc:	48 83 ec 50          	sub    $0x50,%rsp                         # now stack expand 50 bytes
  401100:	49 89 e5             	mov    %rsp,%r13                          # %r13 = %rsp
  401103:	48 89 e6             	mov    %rsp,%rsi                          # %rsi = %rsp
  401106:	e8 51 03 00 00       	call   40145c <read_six_numbers>          # 6 int numbers in |%rsp -> %rsp+24|
  40110b:	49 89 e6             	mov    %rsp,%r14                          # %r14 = %rsp
  40110e:	41 bc 00 00 00 00    	mov    $0x0,%r12d                         # %r12d = 0
  # ------------------ every number different ------------------
  401114:	4c 89 ed             	mov    %r13,%rbp                          # %rbp = &current_num <- init &num1
  401117:	41 8b 45 00          	mov    0x0(%r13),%eax                     # %eax = current_num
  40111b:	83 e8 01             	sub    $0x1,%eax                          # %eax = current_num-1
  40111e:	83 f8 05             	cmp    $0x5,%eax
  401121:	76 05                	jbe    401128 <phase_6+0x34>              # current_num <= 6
  401123:	e8 12 03 00 00       	call   40143a <explode_bomb>
  401128:	41 83 c4 01          	add    $0x1,%r12d                         # %r12d += 1
  40112c:	41 83 fc 06          	cmp    $0x6,%r12d
  401130:	74 21                	je     401153 <phase_6+0x5f>              # [jump if %r12d == 6]
  401132:	44 89 e3             	mov    %r12d,%ebx                         # %rbx = %r12d
  401135:	48 63 c3             	movslq %ebx,%rax                          # %rax = %rbx
  401138:	8b 04 84             	mov    (%rsp,%rax,4),%eax                 # %rax = next_num
  40113b:	39 45 00             	cmp    %eax,0x0(%rbp)                     # cmp currnet_num-next_num
  40113e:	75 05                	jne    401145 <phase_6+0x51>              # current_num != next_num
  401140:	e8 f5 02 00 00       	call   40143a <explode_bomb>
  401145:	83 c3 01             	add    $0x1,%ebx                          # %rbx += 1
  401148:	83 fb 05             	cmp    $0x5,%ebx                          # cmp %rbx-5
  40114b:	7e e8                	jle    401135 <phase_6+0x41>              # [jump if %rbx <= 5]
  40114d:	49 83 c5 04          	add    $0x4,%r13                          # [%rbx > 5] %r13 = &next_num
  401151:	eb c1                	jmp    401114 <phase_6+0x20>
  # --------------------- flip around 3.5 ----------------------
  401153:	48 8d 74 24 18       	lea    0x18(%rsp),%rsi                    # %rsi = %rsp+24
  401158:	4c 89 f0             	mov    %r14,%rax                          # %rax = &current_num <- init num1
  40115b:	b9 07 00 00 00       	mov    $0x7,%ecx                          # %rcx = 7
  401160:	89 ca                	mov    %ecx,%edx                          # %rdx = 7
  401162:	2b 10                	sub    (%rax),%edx                        # %rdx = 7-num1
  401164:	89 10                	mov    %edx,(%rax)                        # num1 = 7-num1
  401166:	48 83 c0 04          	add    $0x4,%rax                          # %rax = next_num
  40116a:	48 39 f0             	cmp    %rsi,%rax                          # cmp addr
  40116d:	75 f1                	jne    401160 <phase_6+0x6c>              # [jump if %rax != %rsi]
  # ---------- store correspinding node addr in stack ----------
  40116f:	be 00 00 00 00       	mov    $0x0,%esi                          # %rsi = 0
  401174:	eb 21                	jmp    401197 <phase_6+0xa3>
  401176:	48 8b 52 08          	mov    0x8(%rdx),%rdx                     # [current_num > 1] %rdx = *(%rdx+8)
  40117a:	83 c0 01             	add    $0x1,%eax                          # %rax += 1
  40117d:	39 c8                	cmp    %ecx,%eax                          # cmp current_num %rax
  40117f:	75 f5                	jne    401176 <phase_6+0x82>
  401181:	eb 05                	jmp    401188 <phase_6+0x94>
  401183:	ba d0 32 60 00       	mov    $0x6032d0,%edx                     # [current_num <= 1] %rdx = 0x6032d0
  401188:	48 89 54 74 20       	mov    %rdx,0x20(%rsp,%rsi,2)             # 0x20(%rsp,%rsi,2) = 0x6032d0
  40118d:	48 83 c6 04          	add    $0x4,%rsi                          # %rsi += 4 
  401191:	48 83 fe 18          	cmp    $0x18,%rsi
  401195:	74 14                	je     4011ab <phase_6+0xb7>
  401197:	8b 0c 34             	mov    (%rsp,%rsi,1),%ecx                 # %rcx = current_num <- init num1, also update to next num
  40119a:	83 f9 01             	cmp    $0x1,%ecx                          # cmp current_num  1
  40119d:	7e e4                	jle    401183 <phase_6+0x8f>              # [jump if current_num <= 1]
  40119f:	b8 01 00 00 00       	mov    $0x1,%eax                          # [current_num > 1] %rax = 1
  4011a4:	ba d0 32 60 00       	mov    $0x6032d0,%edx                     # [current_num > 1] %rdx = 0x6032d0
  4011a9:	eb cb                	jmp    401176 <phase_6+0x82>
  # --------------- reorder the seq among nodes ----------------              
  4011ab:	48 8b 5c 24 20       	mov    0x20(%rsp),%rbx                    # %rbx = node(num1)
  4011b0:	48 8d 44 24 28       	lea    0x28(%rsp),%rax                    # %rax = &node(num2)
  4011b5:	48 8d 74 24 50       	lea    0x50(%rsp),%rsi                    # %rsi = stack_end
  4011ba:	48 89 d9             	mov    %rbx,%rcx                          # %rcx = %rbx = node(first_num) <- init num1
  4011bd:	48 8b 10             	mov    (%rax),%rdx                        # %rdx = *%rax = node(second_num) <- init num2
  4011c0:	48 89 51 08          	mov    %rdx,0x8(%rcx)                     # *(node(first_num)+8) = node(second_num) <- order remap
  4011c4:	48 83 c0 08          	add    $0x8,%rax                          # %rax = &node(next_num(second_num))
  4011c8:	48 39 f0             	cmp    %rsi,%rax
  4011cb:	74 05                	je     4011d2 <phase_6+0xde>
  4011cd:	48 89 d1             	mov    %rdx,%rcx                          # %rcx = node(second_node)
  4011d0:	eb eb                	jmp    4011bd <phase_6+0xc9>
  4011d2:	48 c7 42 08 00 00 00 	movq   $0x0,0x8(%rdx)                     # node(num6)+1 = 0
  4011d9:	00 
  # ------------------------------------------------------------
  4011da:	bd 05 00 00 00       	mov    $0x5,%ebp                          # %rbp = 5, %rbx = current_node
  4011df:	48 8b 43 08          	mov    0x8(%rbx),%rax                     # %rax = current_node + 1
  4011e3:	8b 00                	mov    (%rax),%eax                        # %rax = *(current_node+1)
  4011e5:	39 03                	cmp    %eax,(%rbx)                        # cmp *(current_node+1) *(current_node)
  4011e7:	7d 05                	jge    4011ee <phase_6+0xfa>              # *(current_node+1) <= *(current_node)
  4011e9:	e8 4c 02 00 00       	call   40143a <explode_bomb>
  4011ee:	48 8b 5b 08          	mov    0x8(%rbx),%rbx                     # current_node = current_node+1
  4011f2:	83 ed 01             	sub    $0x1,%ebp
  4011f5:	75 e8                	jne    4011df <phase_6+0xeb>
  4011f7:	48 83 c4 50          	add    $0x50,%rsp
  4011fb:	5b                   	pop    %rbx
  4011fc:	5d                   	pop    %rbp
  4011fd:	41 5c                	pop    %r12
  4011ff:	41 5d                	pop    %r13
  401201:	41 5e                	pop    %r14
  401203:	c3                   	ret    
```
**6 number must be "1" "2" "3" "4" "5" "6", it is the order that matters.**
* 1 => node1: 0x6032d0
* 2 => node2: 0x6032e0
* 3 => node3: 0x6032f0
* 4 => node4: 0x603300
* 5 => node5: 0x603310
* 6 => node6: 0x603320

Originally, we have `node_{x+1} = *(node_{x}+8)`(below code block), but in the `reorder the seq among nodes` I marked, precedure reorder the sequence. And you need to make it fullfill the `*(current_node+1) <= *(current_node)` condition at `0x4011e7`. which compares 4 byte int val at each node address.

* node1: 0x014c -> 5
* node2: 0x00a8 -> 6
* node3: 0x039c -> 1
* node4: 0x02b3 -> 2
* node5: 0x01dd -> 3
* node6: 0x01bb -> 4

```
00000000006032d0 <node1>:
  6032d0:	4c 01 00             	add    %r8,(%rax)
  6032d3:	00 01                	add    %al,(%rcx)
  6032d5:	00 00                	add    %al,(%rax)
  6032d7:	00 e0                	add    %ah,%al
  6032d9:	32 60 00             	xor    0x0(%rax),%ah
  6032dc:	00 00                	add    %al,(%rax)
	...

00000000006032e0 <node2>:
  6032e0:	a8 00                	test   $0x0,%al
  6032e2:	00 00                	add    %al,(%rax)
  6032e4:	02 00                	add    (%rax),%al
  6032e6:	00 00                	add    %al,(%rax)
  6032e8:	f0 32 60 00          	lock xor 0x0(%rax),%ah
  6032ec:	00 00                	add    %al,(%rax)
	...

00000000006032f0 <node3>:
  6032f0:	9c                   	pushf  
  6032f1:	03 00                	add    (%rax),%eax
  6032f3:	00 03                	add    %al,(%rbx)
  6032f5:	00 00                	add    %al,(%rax)
  6032f7:	00 00                	add    %al,(%rax)
  6032f9:	33 60 00             	xor    0x0(%rax),%esp
  6032fc:	00 00                	add    %al,(%rax)
	...

0000000000603300 <node4>:
  603300:	b3 02                	mov    $0x2,%bl
  603302:	00 00                	add    %al,(%rax)
  603304:	04 00                	add    $0x0,%al
  603306:	00 00                	add    %al,(%rax)
  603308:	10 33                	adc    %dh,(%rbx)
  60330a:	60                   	(bad)  
  60330b:	00 00                	add    %al,(%rax)
  60330d:	00 00                	add    %al,(%rax)
	...

0000000000603310 <node5>:
  603310:	dd 01                	fldl   (%rcx)
  603312:	00 00                	add    %al,(%rax)
  603314:	05 00 00 00 20       	add    $0x20000000,%eax
  603319:	33 60 00             	xor    0x0(%rax),%esp
  60331c:	00 00                	add    %al,(%rax)
	...

0000000000603320 <node6>:
  603320:	bb 01 00 00 06       	mov    $0x6000001,%ebx
	...
```

So the seq should be "3 4 5 6 1 2", but remember there exist a flip around, so the ans would be ${\{7-i~|for~i~in~ans\}}$ -> "4 3 2 1 6 5".

I'm pured.

# DO BELOW TO TEST

```
# ans
Border relations with Canada have never been better.
1 2 4 8 16 32
0 207
7 0
Ionuvw
4 3 2 1 6 5
<cr>        # here <cr> means a extra line, of course you can also manually input 6 phase above

# bash 
./bomb < ans
```